In [1]:
import pandas as pd
import numpy as np
import scipy
import torch

In [2]:
import pandas as pd

# Load the item data
item_data_path = 'data/sushi3-2016/sushi3.idata'
item_columns = ['item_id', 'name', 'style', 'major_group', 'minor_group', 'heaviness', 'consumption_frequency', 'normalized_price', 'sell_frequency']
item_df = pd.read_csv(item_data_path, sep='\t', header=None, names=item_columns)

# Filter the item data to include only the 10 items used in the paper
item_set_A_ids = [0, 1, 2, 3, 4, 6, 7, 8, 26, 29]
#item_set_A_ids=[i for i in range(100)]
item_set_A_df = item_df[item_df['item_id'].isin(item_set_A_ids)]

# Preprocess the item features
categorical_features = ['style', 'major_group', 'minor_group']
numerical_features = ['heaviness', 'consumption_frequency', 'normalized_price', 'sell_frequency']

# Convert categorical features to strings to ensure get_dummies works correctly

item_features = pd.get_dummies(item_set_A_df[categorical_features].astype(str))
item_features = pd.concat([item_features, item_set_A_df[numerical_features]], axis=1)

# Display the preprocessed item features
print("Item features shape:", item_features.shape)

# Load the user data
user_data_path = 'data/sushi3-2016/sushi3.udata'
user_columns = ['user_id', 'gender', 'age', 'total_time', 'prefecture_longest', 'region_longest', 'east_west_longest', 'prefecture_current', 'region_current', 'east_west_current', 'prefecture_diff']
user_df = pd.read_csv(user_data_path, sep='\t', header=None, names=user_columns)

# Preprocess the user features
categorical_features_user = ['gender', 'age', 'prefecture_longest', 'region_longest', 'east_west_longest', 'prefecture_current', 'region_current', 'east_west_current']

# Convert categorical features to strings to ensure get_dummies works correctly
user_df[categorical_features_user] = user_df[categorical_features_user].astype(str)

user_features = pd.get_dummies(user_df[categorical_features_user])

# Display the preprocessed user features
print("User features shape:", user_features.shape)


Item features shape: (10, 16)
User features shape: (5000, 128)


# Carregando o arquivo de preferencias e removendo as duas primeiras colunas de metadados

In [3]:
import pandas as pd

# Load and parse the preference order data manually
preference_data_path = 'data/sushi3-2016/sushi3a.5000.10.order'
with open(preference_data_path, 'r') as file:
    lines = file.readlines()

# Remove the first row which contains metadata
lines = lines[1:]

# Split each line into a list of preferences
preference_data = [line.strip().split() for line in lines]

# Convert to a DataFrame
preference_df = pd.DataFrame(preference_data)

# Convert all values to integers
preference_df = preference_df.astype(int)

# Rename columns for clarity
preference_df.columns = [f'pref_{i}' for i in range(preference_df.shape[1])]

# Display the processed preference data

# Remove the first two columns if they contain metadata
preference_df = preference_df.drop(columns=['pref_0', 'pref_1'])

# Rename columns for clarity
preference_df.columns = [f'pref_{i}' for i in range(preference_df.shape[1])]

# Display the cleaned preference data
print(preference_df.shape)

(5000, 10)


In [4]:
def convert_to_pairwise(preference_df):
    pairwise_data = []
    for user_id, preferences in preference_df.iterrows():
        for i in range(len(preferences)):
            for j in range(i + 1, len(preferences)):
                item_i = preferences[i]
                item_j = preferences[j]
                pairwise_data.append([user_id, item_i, item_j])
    pairwise_df = pd.DataFrame(pairwise_data, columns=['user_id', 'item_i', 'item_j'])
    return pairwise_df

# Convert the preference data to pairwise format
pairwise_df = convert_to_pairwise(preference_df)

# Display the processed pairwise preference data
print(pairwise_df.head())
print(pairwise_df.shape)

C:\Users\pedro\AppData\Local\Temp\ipykernel_9128\2437098629.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  item_i = preferences[i]
C:\Users\pedro\AppData\Local\Temp\ipykernel_9128\2437098629.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  item_j = preferences[j]


   user_id  item_i  item_j
0        0       5       0
1        0       5       3
2        0       5       4
3        0       5       6
4        0       5       9
(225000, 3)


In [5]:
#now subsample the pairwise data taking 5 pairs for each of the first 50 users
pairwise_df = pairwise_df[pairwise_df['user_id'] < 100]
pairwise_df = pairwise_df.sample(10*100, random_state=0)

#order df by user_id
pairwise_df = pairwise_df.sort_values('user_id')
pairwise_df

,user_id,item_i,item_j
34,0,6,2
1,0,5,3
31,0,6,8
22,0,3,7
23,0,3,2
...,...,...,...
4484,99,3,5
4496,99,9,5
4457,99,7,3
4464,99,2,1


In [6]:
def get_preference(user_idx, prod_1_idx, prod_2_idx):
    user_pref = preference_df.loc[user_idx]

    for i in range(len(user_pref)):
        if user_pref.iloc[i] == prod_1_idx:
            return 1
        if user_pref.iloc[i] == prod_2_idx:
            return 0

In [7]:
import torch
import numpy as np
import scipy.stats

class GPPrefenceElicitation:
    def __init__(self):
        pass
    
    def fit(self, user_features, item_features, preference_data, max_iter=100):
        self.user_features = user_features
        self.item_features = item_features
        self.preference_data = preference_data
        self.n_users = user_features.shape[0]
        self.n_items = item_features.shape[0]
        self.max_iter=max_iter
        self.maximum_hyperparameters(max_iter=self.max_iter)
        user_covariance = self.kernel_covariance_matrix(user_features, user_features, self.sv_t, self.ls_t)
        item_covariance = self.kernel_covariance_matrix(item_features, item_features, self.sv_k, self.ls_k)
        self.covariance_matrix = torch.kron(user_covariance, item_covariance)
        self.precision_matrix = torch.linalg.inv(self.covariance_matrix+torch.eye(self.n_users*self.n_items)*0.000001)
    
    def grad_log_p_D_given_f(self, f):
        f = torch.tensor(f, dtype=torch.float32)
        grad = torch.zeros_like(f, dtype=torch.float32)
        
        for preference in self.preference_data:
            t, k_1, k_2 = preference
            index_1 = t * self.n_items + k_1
            index_2 = t * self.n_items + k_2
            z = (f[index_1] - f[index_2]) / self.noise_sd
            phi_val = scipy.stats.norm.pdf(z)
            Phi_val = scipy.stats.norm.cdf(z)
            ratio = phi_val / (Phi_val * self.noise_sd)
            grad[index_1] += ratio
            grad[index_2] -= ratio
        
        return grad
        
    def hessian_log_p_D_given_f(self, f):
        hessian = torch.zeros((f.shape[0], f.shape[0]), dtype=torch.float32)
        
        for preference in self.preference_data:
            t, k_1, k_2 = preference
            index_1 = t * self.n_items + k_1
            index_2 = t * self.n_items + k_2
            z = (f[index_1] - f[index_2]) / self.noise_sd
            phi_val = scipy.stats.norm.pdf(z)
            Phi_val = scipy.stats.norm.cdf(z)
            ratio = phi_val / (Phi_val * self.noise_sd)
            
            second_derivative = (z * ratio + ratio**2) / (self.noise_sd ** 2)
            hessian[index_1, index_1] -= second_derivative
            hessian[index_2, index_2] -= second_derivative
            hessian[index_1, index_2] += second_derivative
            hessian[index_2, index_1] += second_derivative
        
        return hessian
    
    def maximum_a_posteriori(self, initial_f, max_iter=100):
        f = torch.tensor(initial_f, dtype=torch.float32)
        for i in range(max_iter):
            hessian = self.hessian_log_p_D_given_f(f)
            precision_matrix_post = hessian + torch.tensor(self.precision_matrix, dtype=torch.float32)
            grad = self.grad_log_p_D_given_f(f)
            hessian_mult_f = hessian @ f
            f = torch.linalg.solve(precision_matrix_post, grad + hessian_mult_f)
        
        return f
    
    def se_kernel(self, x, y, sv, ls):
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        return sv * torch.exp(-0.5 * torch.linalg.norm(x - y) ** 2 / ls ** 2)
    
    def kernel_covariance_matrix(self, x, y, sv, ls):
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        n_x = x.shape[0]
        n_y = y.shape[0]
        covariance_matrix = torch.zeros((n_x, n_y), dtype=torch.float32)
        
        for i in range(n_x):
            for j in range(n_y):
                covariance_matrix[i, j] = self.se_kernel(x[i], y[j], sv, ls)
        
        return covariance_matrix

    def maximum_hyperparameters(self, init_sv_t=1.0, init_ls_t=1.0, init_sv_k=1.0, init_ls_k=1.0, init_noise_sd=0.1, max_iter=100):
        sv_t = torch.tensor(init_sv_t, requires_grad=True, dtype=torch.float32)
        ls_t = torch.tensor(init_ls_t, requires_grad=True, dtype=torch.float32)
        sv_k = torch.tensor(init_sv_k, requires_grad=True, dtype=torch.float32)
        ls_k = torch.tensor(init_ls_k, requires_grad=True, dtype=torch.float32)
        self.noise_sd = torch.tensor(init_noise_sd, requires_grad=True, dtype=torch.float32)
        optimizer = torch.optim.Adam([sv_t, ls_t, sv_k, ls_k, self.noise_sd])
        identity = torch.eye(self.n_users * self.n_items, dtype=torch.float32)
        for i in range(max_iter):
            optimizer.zero_grad()
            user_covariance = self.kernel_covariance_matrix(self.user_features, self.user_features, sv_t, ls_t)
            item_covariance = self.kernel_covariance_matrix(self.item_features, self.item_features, sv_k, ls_k)
            self.covariance_matrix = torch.kron(user_covariance, item_covariance)
            self.precision_matrix = torch.inverse(self.covariance_matrix + torch.eye(self.n_users * self.n_items, dtype=torch.float32) * 0.000001)
            
            with torch.no_grad():
                f = self.maximum_a_posteriori(np.zeros(self.n_users * self.n_items), max_iter=self.max_iter)
                hessian = self.hessian_log_p_D_given_f(f)
                grad = self.grad_log_p_D_given_f(f)
            
            f = f.reshape(1, -1)
            sum_log_cdf = 0
            for preference in self.preference_data:
                t, k_1, k_2 = preference
                index_1 = t * self.n_items + k_1
                index_2 = t * self.n_items + k_2
                sum_log_cdf += torch.log(torch.tensor(scipy.stats.norm.cdf((f[0, index_1] - f[0, index_2]) / self.noise_sd.detach().numpy()), dtype=torch.float32))
            
            loss = 0.5 * torch.logdet(self.covariance_matrix @ hessian + identity) + 0.5 * f @ self.precision_matrix @ f.T - sum_log_cdf
            loss = loss.sum()  # Ensure the loss is a scalar
            loss.backward()
            optimizer.step()
        
        self.sv_t = sv_t.item()
        self.ls_t = ls_t.item()
        self.sv_k = sv_k.item()
        self.ls_k = ls_k.item()
        self.noise_sd = self.noise_sd.item()
    
    def predictive_mean_and_variance(self, user_features, item_features):
        user_covariance = self.kernel_covariance_matrix(user_features, self.user_features, self.sv_t, self.ls_t)
        item_covariance = self.kernel_covariance_matrix(item_features, self.item_features, self.sv_k, self.ls_k)
        covariance_matrix = torch.kron(user_covariance, item_covariance)
        predictive_mean = item_covariance @ self.precision_matrix @ torch.tensor(self.preference_data, dtype=torch.float32)
        predictive_variance = item_covariance - item_covariance @ self.precision_matrix @ item_covariance
        
        return predictive_mean, predictive_variance

    def joint_predictive_mean_and_variance(self, f_hat, hessian, t_features, x_1_features, x_2_features):
        k_t_star = torch.zeros(self.n_users, 1)
        for i in range(self.n_users):
            k_t_star[i] = self.se_kernel(t_features, self.user_features[i], self.sv_t, self.ls_t)
        
        k_x_star = torch.zeros(self.n_items, 2)
        for i in range(self.n_items):
            k_x_star[i, 0] = self.se_kernel(x_1_features, self.item_features[i], self.sv_k, self.ls_k)
            k_x_star[i, 1] = self.se_kernel(x_2_features, self.item_features[i], self.sv_k, self.ls_k)
        
        kernel_t = self.kernel_covariance_matrix(t_features, t_features, self.sv_t, self.ls_t)
        Sigma_star = torch.zeros(2, 2)
        Sigma_star[0, 0] = kernel_t * self.kernel_covariance_matrix(x_1_features, x_1_features, self.sv_k, self.ls_k)
        Sigma_star[1, 1] = kernel_t * self.kernel_covariance_matrix(x_2_features, x_2_features, self.sv_k, self.ls_k)
        Sigma_star[0, 1] = kernel_t * self.kernel_covariance_matrix(x_1_features, x_2_features, self.sv_k, self.ls_k)
        Sigma_star[1, 0] = kernel_t * self.kernel_covariance_matrix(x_2_features, x_1_features, self.sv_k, self.ls_k)
        
        k_star = torch.kron(k_t_star, k_x_star)
        
        predictive_mean = k_star.T @ self.precision_matrix @ f_hat
        predictive_variance = Sigma_star - k_star.T @ torch.inverse(self.covariance_matrix + torch.inverse(hessian)) @ k_star
        
        return predictive_mean, predictive_variance
    
    def expected_improvement(self, t_features, x_features, f_hat, hessian, f_best=None):
        if f_best is None:
            f_best = torch.max(f_hat)
        
        predictive_mean, predictive_variance = self.predictive_mean_and_variance(f_hat, hessian, t_features, x_features)
        
        pred_sd = torch.sqrt(predictive_variance)
        z_prime = (predictive_mean - f_best) / pred_sd
        
        return pred_sd * (z_prime * scipy.stats.norm.cdf(z_prime) + scipy.stats.norm.pdf(z_prime))
    
    def maximum_expected_improvement(self, t_features, item_features, f_hat, hessian, f_best=None):
        maximum_ei = -np.inf
        
        for x in item_features:
            ei = self.expected_improvement(t_features, x, f_hat, hessian, f_best)
            if ei > maximum_ei:
                maximum_ei = ei
        
        return maximum_ei
    
    def prob_i_over_j(self, t_features, x_1_index, x_2_index, f_hat, hessian):
        predictive_mean, predictive_variance = self.joint_predictive_mean_and_variance(f_hat, hessian, t_features, self.item_features[x_1_index], self.item_features[x_2_index])
        
        numerator = predictive_mean[0] - predictive_mean[1]
        denominator = predictive_variance[0, 0] - predictive_variance[1, 1] - 2 * predictive_variance[0, 1] - self.noise_sd ** 2
        
        return scipy.stats.norm.cdf(numerator / denominator)

    def expected_value_of_information(self, t_index, x_1_index, x_2_index, f_hat, hessian, f_best=None):
        t_features = self.user_features[t_index]
        maximum_ei = self.maximum_expected_improvement(t_features, self.item_features, f_hat, hessian, f_best)
        
        preference_data_1_2 = self.preference_data + [[t_index, x_1_index, x_2_index]]
        f_hat_1_2 = self.maximum_a_posteriori(f_hat, preference_data_1_2)
        hessian_1_2 = self.hessian_log_p_D_given_f(f_hat_1_2, preference_data_1_2)
        
        preference_data_2_1 = self.preference_data + [[t_index, x_2_index, x_1_index]]
        f_hat_2_1 = self.maximum_a_posteriori(f_hat, preference_data_2_1)
        hessian_2_1 = self.hessian_log_p_D_given_f(f_hat_2_1, preference_data_2_1)
        
        mei_1_2 = self.maximum_expected_improvement(t_features, self.item_features, f_hat_1_2, hessian_1_2, f_best)
        mei_2_1 = self.maximum_expected_improvement(t_features, self.item_features, f_hat_2_1, hessian_2_1, f_best)
        
        prob_1_2 = self.prob_i_over_j(t_features, x_1_index, x_2_index, f_hat, hessian)
        prob_2_1 = 1 - prob_1_2
        
        return -maximum_ei + prob_1_2 * mei_1_2 + prob_2_1 * mei_2_1

    def preference_elicitation(self, t_index, num_queries):
        # Make a copy
        pref_data = self.preference_data.copy()
        f_hat = self.maximum_a_posteriori(np.zeros(self.n_users * self.n_items))
        hessian = self.hessian_log_p_D_given_f(f_hat, pref_data)
        pred_hist = []
        
        for i in range(num_queries):
            max_evoi = -np.inf
            for x_1 in range(self.n_items):
                for x_2 in range(x_1 + 1, self.n_items):
                    evoi = self.expected_value_of_information(t_index, x_1, x_2, f_hat, hessian)
                    if evoi > max_evoi:
                        max_evoi = evoi
                        max_x_1 = x_1
                        max_x_2 = x_2
            
            if get_preference(t_index, max_x_1, max_x_2):
                pref_data.append([t_index, max_x_1, max_x_2])
            else:
                pref_data.append([t_index, max_x_2, max_x_1])
            
            f_hat = self.maximum_a_posteriori(f_hat, pref_data)
            pred_hist.append(f_hat[t_index * self.n_items:(t_index + 1) * self.n_items])
            hessian = self.hessian_log_p_D_given_f(f_hat, pref_data)
        
        return pred_hist

# Example usage
# model = GPPrefenceElicitation()
# user_features = np.random.rand(50, 5)  # Replace with actual user features
# item_features = np.random.rand(10, 5)  # Replace with actual item features
# preference_data = np.array([[0, 1, 2], [1, 3, 4], [2, 5, 6]])  # Replace with actual preference data

# model.fit(user_features, item_features, preference_data)

In [8]:
user_features_v=torch.tensor(user_features.values, dtype=torch.float32)
item_features_v=torch.tensor(item_features.values, dtype=torch.float32)

# model = GPPrefenceElicitation()
# model.fit(user_features_v[:100], item_features_v, pairwise_df.values,max_iter=2)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\pedro\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\pedro\Desktop\GP-PreferenceElicitation\.venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\pedro\Desktop\GP-PreferenceElicitation\.venv\lib\site-packages\traitle

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import torch

# Substitua pelas suas variáveis reais
# user_features_v = np.random.rand(50, 5)  # Exemplo de dados de características dos usuários
# item_features_v = np.random.rand(10, 5)  # Exemplo de dados de características dos itens

# # Exemplo de dados de preferências pareadas como DataFrame
# pairwise_df = pd.DataFrame({
#     'user_id': np.repeat(np.arange(50), 10),
#     'item_1': np.tile(np.arange(10), 50),
#     'item_2': np.tile((np.arange(10) + 1) % 10, 50)
# })

# Instancie seu modelo
model = GPPrefenceElicitation()

# Defina o número de divisões para a validação cruzada
kf = KFold(n_splits=10)

# Lista para armazenar as perdas normalizadas para diferentes números de consultas
all_normalized_losses = []

prediction_histories = {}

# Realize a validação cruzada
for train_index, test_index in kf.split(pairwise_df['user_id'].unique()):
    # Ajuste os índices para incluir todos os 10 pares de cada usuário
    train_users = pairwise_df['user_id'].isin(train_index)
    test_users = pairwise_df['user_id'].isin(test_index)

    # Divida os dados em treino e teste
    train_pairwise = pairwise_df[train_users].values
    test_pairwise = pairwise_df[test_users].values
    
    # Ajuste o modelo aos dados de treino
    model.fit(user_features_v, item_features_v, train_pairwise, max_iter=2)
        
    # Realize a elicitação para o conjunto de teste
    for t_index in test_index:
        prediction_histories[t_index] = model.preference_elicitation(t_index, num_queries=15)

# # Combine e analise os resultados
# all_normalized_losses = np.array(all_normalized_losses)
# mean_losses = np.mean(all_normalized_losses, axis=0)
# std_losses = np.std(all_normalized_losses, axis=0)

# for i, num_queries in enumerate(query_counts):
#     print(f"Mean normalized loss for {num_queries} queries:", mean_losses[i])
#     print(f"Standard deviation of normalized loss for {num_queries} queries:", std_losses[i])

C:\Users\pedro\AppData\Local\Temp\ipykernel_9128\3409976812.py:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)
C:\Users\pedro\AppData\Local\Temp\ipykernel_9128\3409976812.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32)
C:\Users\pedro\AppData\Local\Temp\ipykernel_9128\3409976812.py:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32)
C:\Users\pedro\AppData\Local\Temp\ipykernel_9128\3409976812.py:72: UserWarning: To